In [10]:
from exif import Image
with open("./images/duma.jpg", "rb") as image_file:
    my_image = Image(image_file)
# Open 
my_image.has_exif
dir(my_image)
# This reads the images metadata

['_exif_ifd_pointer',
 '_segments',
 'aperture_value',
 'color_space',
 'compression',
 'datetime_digitized',
 'datetime_original',
 'delete',
 'delete_all',
 'digital_zoom_ratio',
 'exif_version',
 'exposure_bias_value',
 'exposure_mode',
 'exposure_program',
 'exposure_time',
 'f_number',
 'flash',
 'focal_length',
 'focal_length_in_35mm_film',
 'get',
 'get_all',
 'get_file',
 'get_thumbnail',
 'has_exif',
 'image_height',
 'image_unique_id',
 'image_width',
 'jpeg_interchange_format',
 'jpeg_interchange_format_length',
 'list_all',
 'make',
 'max_aperture_value',
 'metering_mode',
 'model',
 'offset_time',
 'offset_time_original',
 'orientation',
 'photographic_sensitivity',
 'pixel_x_dimension',
 'pixel_y_dimension',
 'resolution_unit',
 'scene_capture_type',
 'shutter_speed_value',
 'software',
 'white_balance',
 'x_resolution',
 'y_and_c_positioning',
 'y_resolution']

In [ ]:
from exif import Image
import os

directory = "images"

def img_metachecker():
    for image in os.listdir(directory):
        try:
            i = os.path.join(directory, image)
            if os.path.isfile(i):
                # print(i)
                curr_image = Image(i)
                print(dir(curr_image))
        except:
            pass

img_metachecker()

# def meta_checker():
#     with open(i, "rb") as image_dir:
#         my_image = Image(image_dir)
#     # Open 
#         # my_image.has_exif
#         dir(my_image)
    # This reads the images metadata

# meta_checker()

In [13]:
directory = "images"

def img_filepaths():
    for image in os.listdir(directory):
        i = os.path.join(directory, image)
        if os.path.isfile(i):
            return i
            # curr_image = Image(i)
            # dir(curr_image)

# img_filepaths()

# def meta_checker():
#     img_filepaths()
#     with open(directory, "rb") as image_dir:
#         my_image = Image(image_dir)
#     # Open 
#         # my_image.has_exif
#         dir(my_image)
#     # This reads the images metadata

# meta_checker()

In [27]:
#import Pillow
from PIL import Image
from PIL.ExifTags import TAGS
from csv import DictWriter

#call filepaths from img_filepaths function and open w/ pillow
image_names = img_filepaths()
image = Image.open(image_names)

#define dict
info_dict = {
    "Filename": image.filename,
    "Image Size": image.size,
    "Image Height": image.height,
    "Image Width": image.width,
    "Image Format": image.format,
}

#prints info_dict
for key,value in info_dict.items():
    print(f"{key:25}: {value}")

#call getexif()
exifdata = image.getexif()

#maps pillow tags onto id fields and prints
header = []
file_dict = {}
for tag_id in exifdata:
    # get the tag name, instead of human unreadable tag id
    tag = TAGS.get(tag_id, tag_id)
    data = exifdata.get(tag_id)
    # decode bytes 
    if isinstance(data, bytes):
        data = data.decode()
    #add tags into header list
    header.append(tag)
    print(data)
    # print(f"{tag:25}: {data}")

print(header)


# meta_df = pd.DataFrame(columns= header)
# for i in data

# with open ('./data/meta_data.csv', 'w') as data_file:
#      metadata_writer = DictWriter(data_file, fieldnames=header, dialect="unix")
#      metadata_writer.writeheader()
#      for row in rows:
#          metadata_writer.writerow(row)



Filename                 : images/20220308_153843.jpg
Image Size               : (4032, 3024)
Image Height             : 3024
Image Width              : 4032
Image Format             : JPEG
4032
3024
2
228
samsung
SM-G991U1
G991U1UES5BVA6
1
2022:03:08 15:38:43
1
72.0
72.0
['ImageWidth', 'ImageLength', 'ResolutionUnit', 'ExifOffset', 'Make', 'Model', 'Software', 'Orientation', 'DateTime', 'YCbCrPositioning', 'XResolution', 'YResolution']
